In [37]:
import pandas as pd
import numpy as np
import os
import sys
import pickle


In [38]:
def load_input_data():

    test_path = "C:/Users/diego/OneDrive/Escritorio/mlops_projects/mlops/rurall_prueba/data/drugs_test.csv"
    try:
        # Cargar el CSV en un DataFrame de pandas
        df = pd.read_csv(test_path)

        return df
    
    except FileNotFoundError:
        print("El archivo CSV no se encontró en la ubicación especificada.")
        
        return None

In [39]:
df = load_input_data()

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3671 entries, 0 to 3670
Data columns (total 13 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   drug_id                          3671 non-null   object
 1   description                      3671 non-null   object
 2   administrative_status            3671 non-null   object
 3   marketing_status                 3671 non-null   object
 4   approved_for_hospital_use        3671 non-null   object
 5   reimbursement_rate               3671 non-null   object
 6   dosage_form                      3671 non-null   object
 7   route_of_administration          3671 non-null   object
 8   marketing_authorization_status   3671 non-null   object
 9   marketing_declaration_date       3671 non-null   int64 
 10  marketing_authorization_date     3671 non-null   int64 
 11  marketing_authorization_process  3671 non-null   object
 12  pharmaceutical_companies         3

In [ ]:
#df = df.sample(5)

In [5]:
import requests
import logging
from datetime import datetime

In [41]:
import sys
import os
from dataclasses import dataclass

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from typing import Union
from sklearn.model_selection import train_test_split
from datetime import datetime
import pandas as pd
import numpy as np
import pickle
import logging


@dataclass
class DataTransformationConfig:
    pass

class PreprocessData:
   
    def remove_duplicates(self, df: pd.DataFrame) -> pd.DataFrame:
        try:
            df.drop_duplicates(inplace=True)
            return df
        
        except Exception as e:
            logging.error(e)
            raise e
    

    def impute_missing_values(self, df: pd.DataFrame) -> pd.DataFrame:

        try:
            # numerical columns
            categorical_cols = df.select_dtypes(include='object').columns
            categorical_imputer = SimpleImputer(strategy='most_frequent')
            df[categorical_cols] = categorical_imputer.fit_transform(df[categorical_cols])
            
            # categorical columns
            numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
            numeric_imputer = SimpleImputer(strategy='mean')
            df[numeric_cols] = numeric_imputer.fit_transform(df[numeric_cols])
            
            return df
        
        except Exception as e:
            logging.error(e)
            raise e

    

class FeatureEngineering:

    def feature_enrichment(self, df: pd.DataFrame) -> pd.DataFrame:

        features = pd.read_csv('drug_label_feature_eng.csv')
        routes = pd.read_csv('global_route_admon.csv')
        aditives = pd.read_csv('final_drugs_label.csv')
        
        merge1 = pd.merge(df, aditives, how='left', on='drug_id')
        merge2 = pd.merge(merge1, routes, how='left', on='route_of_administration')
        merge3 = pd.merge(merge2, features, how='left', on='description')
        
        return merge3
    
    def format_date(self, date_str):
        date_obj = datetime.strptime(date_str, '%Y%m%d')
        return date_obj.strftime('%Y-%m-%d')
    
    
    def feature_engineering(self, df: pd.DataFrame) -> pd.DataFrame:

     
        class_frequencies = df['pharmaceutical_companies'].value_counts()
        cumulative_proportion = class_frequencies.cumsum() / class_frequencies.sum()
        labels = []
        
        for company in df['pharmaceutical_companies']:
            proportion = cumulative_proportion[company]
            if proportion <= 0.4:
                labels.append('share_40%')
            elif proportion <= 0.6:
                labels.append('share_60%')
            elif proportion <= 0.8:
                labels.append('share_80%')
            elif proportion <= 0.95:
                labels.append('share_95%')
            else:
                labels.append('share_100%')
                
        # Agregar las etiquetas al DataFrame
        df['pareto_companies'] = labels
        
        # Convertir formatos de fecha
        df['marketing_declaration_date'] = df['marketing_declaration_date'].astype(str)
        df['marketing_authorization_date'] = df['marketing_authorization_date'].astype(str)
    
        df['declaration_date'] = df['marketing_declaration_date'].apply(self.format_date)
        df['authorization_date'] = df['marketing_authorization_date'].apply(self.format_date)
    
         # Calcular la diferencia en años
        df['authorization_date'] = pd.to_datetime(df['authorization_date'])
        df['declaration_date'] = pd.to_datetime(df['declaration_date'])
    
        df['year_of_wait'] = (df['declaration_date'].dt.year - df['authorization_date'].dt.year)
    
        return df


    def normalize_numeric_features(self, df: pd.DataFrame) -> pd.DataFrame:

        try:
            numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
            scaler = MinMaxScaler()
            df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
            return df
        
        except Exception as e:
            logging.error(e)
            raise e
        

    def drop_columns(self, df: pd.DataFrame): # -> pd.DataFrame:
        try:
            df = df.drop(["authorization_date", "marketing_authorization_date",
                      "declaration_date", "marketing_declaration_date",
                      "pharmaceutical_companies", "dosage_form", "drug_id",
                      "description", "route_of_administration", "active_ingredients"], axis=1)
            return df
        
        except Exception as e:
            logging.error(e)
            raise e 


class Preprocessor(PreprocessData, FeatureEngineering):

    def __init__(self):
        self.data_transformation_config = DataTransformationConfig()

    def format_dtypes(self,df):
        df['drug_id'] = df['drug_id'].astype(str)
        df['description'] = df['description'].astype(str)
        df['administrative_status'] = df['administrative_status'].astype(str)
        df['marketing_status'] = df['marketing_status'].astype(str)
        df['approved_for_hospital_use'] = df['approved_for_hospital_use'].astype(str)
        df['reimbursement_rate'] = df['reimbursement_rate'].astype(str)
        df['dosage_form'] = df['dosage_form'].astype(str)
        df['route_of_administration'] = df['route_of_administration'].astype(str)
        df['marketing_authorization_status'] = df['marketing_authorization_status'].astype(str)
        df['marketing_declaration_date'] = df['marketing_declaration_date'].astype(int)
        df['marketing_authorization_date'] = df['marketing_authorization_date'].astype(int)
        df['marketing_authorization_process'] = df['marketing_authorization_process'].astype(str)
        df['pharmaceutical_companies'] = df['pharmaceutical_companies'].astype(str)
        
        return df

    def preprocess_dataframe(self, df):

        df = self.format_dtypes(df)
        enriched_df = self.feature_enrichment(df)
        engineered_df = self.feature_engineering(enriched_df)
        normalized_df = self.normalize_numeric_features(engineered_df)
        dropped_df = self.drop_columns(normalized_df)
        preprocessed_df = self.impute_missing_values(dropped_df)

        return preprocessed_df 




## MODEL TEST

In [42]:
import os
from typing import List, Dict
import logging


In [43]:
input_df = df

In [44]:
input_df.head(5)

,drug_id,description,administrative_status,marketing_status,approved_for_hospital_use,reimbursement_rate,dosage_form,route_of_administration,marketing_authorization_status,marketing_declaration_date,marketing_authorization_date,marketing_authorization_process,pharmaceutical_companies
0,0_test,plaquette(s) thermoformée(s) polyamide alumini...,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,comprimé gastro-résistant(e),orale,Autorisation active,20130101,20080101,Procédure de reconnaissance mutuelle,TEVA SANTE
1,1_test,plaquette(s) thermoformée(s) PVC PVDC aluminiu...,Présentation active,Déclaration de commercialisation,non,30%,capsule,vaginale,Autorisation active,20120101,19970101,Procédure nationale,INNOTECH INTERNATIONAL
2,2_test,plaquette(s) thermoformée(s) PVC-Aluminium de ...,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,comprimé,orale,Autorisation abrogée,20130101,20060101,Procédure nationale,ACTAVIS FRANCE
3,3_test,5 cartouche(s) en verre de 3 ml dans stylo pré...,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,suspension injectable,sous-cutanée,Autorisation active,20120101,20000101,Procédure centralisée,SANOFI AVENTIS (ALLEMAGNE)
4,4_test,5 poche(s) à 3 compartiments polyamide polypro...,Présentation active,Déclaration de commercialisation,oui,65%,émulsion pour perfusion,intraveineuse,Autorisation active,20110101,20110101,Procédure décentralisée,B BRAUN MELSUNGEN


In [45]:
input_list = input_df.to_dict(orient='records')

In [46]:
input_list

[{'drug_id': '0_test',
  'description': 'plaquette(s) thermoformée(s) polyamide aluminium PVC-Aluminium de 14 comprimé(s)',
  'administrative_status': 'Présentation active',
  'marketing_status': "Déclaration d'arrêt de commercialisation",
  'approved_for_hospital_use': 'non',
  'reimbursement_rate': '65%',
  'dosage_form': 'comprimé gastro-résistant(e)',
  'route_of_administration': 'orale',
  'marketing_authorization_status': 'Autorisation active',
  'marketing_declaration_date': 20130101,
  'marketing_authorization_date': 20080101,
  'marketing_authorization_process': 'Procédure de reconnaissance mutuelle',
  'pharmaceutical_companies': ' TEVA SANTE'},
 {'drug_id': '1_test',
  'description': 'plaquette(s) thermoformée(s) PVC PVDC aluminium de 6 capsule(s)',
  'administrative_status': 'Présentation active',
  'marketing_status': 'Déclaration de commercialisation',
  'approved_for_hospital_use': 'non',
  'reimbursement_rate': '30%',
  'dosage_form': 'capsule',
  'route_of_administrati

In [ ]:
#output_list = [input_data.dict() for input_data in input_list]

In [47]:
output_list = input_list.copy()

In [48]:
input_df = pd.DataFrame(output_list)

In [49]:
input_df

,drug_id,description,administrative_status,marketing_status,approved_for_hospital_use,reimbursement_rate,dosage_form,route_of_administration,marketing_authorization_status,marketing_declaration_date,marketing_authorization_date,marketing_authorization_process,pharmaceutical_companies
0,0_test,plaquette(s) thermoformée(s) polyamide alumini...,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,comprimé gastro-résistant(e),orale,Autorisation active,20130101,20080101,Procédure de reconnaissance mutuelle,TEVA SANTE
1,1_test,plaquette(s) thermoformée(s) PVC PVDC aluminiu...,Présentation active,Déclaration de commercialisation,non,30%,capsule,vaginale,Autorisation active,20120101,19970101,Procédure nationale,INNOTECH INTERNATIONAL
2,2_test,plaquette(s) thermoformée(s) PVC-Aluminium de ...,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,comprimé,orale,Autorisation abrogée,20130101,20060101,Procédure nationale,ACTAVIS FRANCE
3,3_test,5 cartouche(s) en verre de 3 ml dans stylo pré...,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,suspension injectable,sous-cutanée,Autorisation active,20120101,20000101,Procédure centralisée,SANOFI AVENTIS (ALLEMAGNE)
4,4_test,5 poche(s) à 3 compartiments polyamide polypro...,Présentation active,Déclaration de commercialisation,oui,65%,émulsion pour perfusion,intraveineuse,Autorisation active,20110101,20110101,Procédure décentralisée,B BRAUN MELSUNGEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3666,3666_test,6 ampoule(s) en verre brun de 2 ml,Présentation active,Déclaration de commercialisation,oui,30%,solution injectable,intramusculaire,Autorisation active,20080101,20060101,Procédure nationale,PHARMY II
3667,3667_test,1 seringue(s) préremplie(s) en verre de 20 ml,Présentation active,Déclaration de commercialisation,oui,65%,solution injectable,intra-articulaire,Autorisation active,20030101,20020101,Procédure nationale,GUERBET
3668,3668_test,plaquette(s) aluminium polyamide PVC-Aluminium...,Présentation active,Déclaration de commercialisation,oui,65%,comprimé gastro-résistant(e),orale,Autorisation active,20120101,20090101,Procédure décentralisée,RANBAXY PHARMACIE GENERIQUES
3669,3669_test,1 flacon(s) en verre de 100 ml,Présentation active,Déclaration de commercialisation,oui,65%,solution pour perfusion,intraveineuse,Autorisation active,20140101,20130101,Procédure nationale,BIOGARAN


In [50]:
original_df = input_df.copy()

In [51]:
preprocessor = Preprocessor()

input_df_proc = preprocessor.preprocess_dataframe(input_df)

In [52]:
input_df_proc 

,administrative_status,marketing_status,approved_for_hospital_use,reimbursement_rate,marketing_authorization_status,marketing_authorization_process,group_related,num_act_ingr,main_route_of_administration,label_plaquette,...,count_pilulier,count_sachet,count_comprime,count_gelule,count_film,count_poche,count_capsule,count_ml,pareto_companies,year_of_wait
0,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,Autorisation active,Procédure de reconnaissance mutuelle,Otras,0.030303,orale,1.000000,...,0.000000,0.00000,0.051852,0.000000,0.000000,0.000000,0.000000,0.000000,share_40%,0.712871
1,Présentation active,Déclaration de commercialisation,non,30%,Autorisation active,Procédure nationale,Antibióticos y Antimicrobianos,0.060606,Autre,0.716396,...,0.003187,0.00534,0.103374,0.020305,0.000869,0.000913,0.004369,0.007459,share_100%,0.811881
2,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,Autorisation abrogée,Procédure nationale,Antihipertensivos y Cardiovasculares,0.000000,orale,1.000000,...,0.000000,0.00000,0.311111,0.000000,0.000000,0.000000,0.000000,0.000000,share_80%,0.732673
3,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,Autorisation active,Procédure centralisée,Antidiabéticos,0.000000,Autre,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001500,share_95%,0.782178
4,Présentation active,Déclaration de commercialisation,oui,65%,Autorisation active,Procédure décentralisée,Hormonas y Moduladores Hormonales,0.909091,Intraveineuse,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.125000,0.000000,0.937500,share_100%,0.663366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3666,Présentation active,Déclaration de commercialisation,oui,30%,Autorisation active,Procédure nationale,Antiinflamatorios y Analgésicos,0.000000,Intramusculaire,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001000,share_100%,0.683168
3667,Présentation active,Déclaration de commercialisation,oui,65%,Autorisation active,Procédure nationale,Inmunosupresores y Moduladores del Sistema Inmune,0.000000,Autre,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,share_80%,0.673267
3668,Présentation active,Déclaration de commercialisation,oui,65%,Autorisation active,Procédure décentralisée,Otras,0.030303,orale,1.000000,...,0.000000,0.00000,0.103704,0.000000,0.000000,0.000000,0.000000,0.000000,share_60%,0.693069
3669,Présentation active,Déclaration de commercialisation,oui,65%,Autorisation active,Procédure nationale,Otras,0.030303,Intraveineuse,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,share_40%,0.673267


In [53]:
input_df_proc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3671 entries, 0 to 3670
Data columns (total 38 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   administrative_status            3671 non-null   object 
 1   marketing_status                 3671 non-null   object 
 2   approved_for_hospital_use        3671 non-null   object 
 3   reimbursement_rate               3671 non-null   object 
 4   marketing_authorization_status   3671 non-null   object 
 5   marketing_authorization_process  3671 non-null   object 
 6   group_related                    3671 non-null   object 
 7   num_act_ingr                     3671 non-null   float64
 8   main_route_of_administration     3671 non-null   object 
 9   label_plaquette                  3671 non-null   float64
 10  label_ampoule                    3671 non-null   float64
 11  label_flacon                     3671 non-null   float64
 12  label_tube          

In [54]:
import os
import pickle

def cargar_modelo():
    ruta_proyecto = os.path.dirname(os.getcwd())  # Obtener la ruta del directorio principal del proyecto
    ruta_modelo = os.path.join(ruta_proyecto, 'api_artifacts', 'model.pkl')
    try:
        with open(ruta_modelo, 'rb') as modelo_archivo:
            modelo = pickle.load(modelo_archivo)
            return modelo
    except Exception as e:
        print("Error al cargar el modelo:", e)
        return None

# Llamada a la función para cargar el modelo
model = cargar_modelo()

if model is not None:
    print("Modelo cargado exitosamente.")
else:
    print("No se pudo cargar el modelo.")

Modelo cargado exitosamente.


In [21]:
model

In [55]:
predictions  = model.predict(input_df_proc)

In [56]:
predictions

array([14.17137206, 18.13348466, 42.76168339, ..., 23.53294302,
       33.04058429, 25.24960478])

In [57]:
original_df['prediction'] = predictions

In [58]:
original_df

,drug_id,description,administrative_status,marketing_status,approved_for_hospital_use,reimbursement_rate,dosage_form,route_of_administration,marketing_authorization_status,marketing_declaration_date,marketing_authorization_date,marketing_authorization_process,pharmaceutical_companies,prediction
0,0_test,plaquette(s) thermoformée(s) polyamide alumini...,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,comprimé gastro-résistant(e),orale,Autorisation active,20130101,20080101,Procédure de reconnaissance mutuelle,TEVA SANTE,14.171372
1,1_test,plaquette(s) thermoformée(s) PVC PVDC aluminiu...,Présentation active,Déclaration de commercialisation,non,30%,capsule,vaginale,Autorisation active,20120101,19970101,Procédure nationale,INNOTECH INTERNATIONAL,18.133485
2,2_test,plaquette(s) thermoformée(s) PVC-Aluminium de ...,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,comprimé,orale,Autorisation abrogée,20130101,20060101,Procédure nationale,ACTAVIS FRANCE,42.761683
3,3_test,5 cartouche(s) en verre de 3 ml dans stylo pré...,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,suspension injectable,sous-cutanée,Autorisation active,20120101,20000101,Procédure centralisée,SANOFI AVENTIS (ALLEMAGNE),102.173439
4,4_test,5 poche(s) à 3 compartiments polyamide polypro...,Présentation active,Déclaration de commercialisation,oui,65%,émulsion pour perfusion,intraveineuse,Autorisation active,20110101,20110101,Procédure décentralisée,B BRAUN MELSUNGEN,129.046569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3666,3666_test,6 ampoule(s) en verre brun de 2 ml,Présentation active,Déclaration de commercialisation,oui,30%,solution injectable,intramusculaire,Autorisation active,20080101,20060101,Procédure nationale,PHARMY II,5.305203
3667,3667_test,1 seringue(s) préremplie(s) en verre de 20 ml,Présentation active,Déclaration de commercialisation,oui,65%,solution injectable,intra-articulaire,Autorisation active,20030101,20020101,Procédure nationale,GUERBET,51.013442
3668,3668_test,plaquette(s) aluminium polyamide PVC-Aluminium...,Présentation active,Déclaration de commercialisation,oui,65%,comprimé gastro-résistant(e),orale,Autorisation active,20120101,20090101,Procédure décentralisée,RANBAXY PHARMACIE GENERIQUES,23.532943
3669,3669_test,1 flacon(s) en verre de 100 ml,Présentation active,Déclaration de commercialisation,oui,65%,solution pour perfusion,intraveineuse,Autorisation active,20140101,20130101,Procédure nationale,BIOGARAN,33.040584


In [59]:
prediction_list = original_df.to_dict(orient='records')

In [60]:
predictions_df = pd.DataFrame(prediction_list)

In [61]:
predictions_df

,drug_id,description,administrative_status,marketing_status,approved_for_hospital_use,reimbursement_rate,dosage_form,route_of_administration,marketing_authorization_status,marketing_declaration_date,marketing_authorization_date,marketing_authorization_process,pharmaceutical_companies,prediction
0,0_test,plaquette(s) thermoformée(s) polyamide alumini...,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,comprimé gastro-résistant(e),orale,Autorisation active,20130101,20080101,Procédure de reconnaissance mutuelle,TEVA SANTE,14.171372
1,1_test,plaquette(s) thermoformée(s) PVC PVDC aluminiu...,Présentation active,Déclaration de commercialisation,non,30%,capsule,vaginale,Autorisation active,20120101,19970101,Procédure nationale,INNOTECH INTERNATIONAL,18.133485
2,2_test,plaquette(s) thermoformée(s) PVC-Aluminium de ...,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,comprimé,orale,Autorisation abrogée,20130101,20060101,Procédure nationale,ACTAVIS FRANCE,42.761683
3,3_test,5 cartouche(s) en verre de 3 ml dans stylo pré...,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,suspension injectable,sous-cutanée,Autorisation active,20120101,20000101,Procédure centralisée,SANOFI AVENTIS (ALLEMAGNE),102.173439
4,4_test,5 poche(s) à 3 compartiments polyamide polypro...,Présentation active,Déclaration de commercialisation,oui,65%,émulsion pour perfusion,intraveineuse,Autorisation active,20110101,20110101,Procédure décentralisée,B BRAUN MELSUNGEN,129.046569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3666,3666_test,6 ampoule(s) en verre brun de 2 ml,Présentation active,Déclaration de commercialisation,oui,30%,solution injectable,intramusculaire,Autorisation active,20080101,20060101,Procédure nationale,PHARMY II,5.305203
3667,3667_test,1 seringue(s) préremplie(s) en verre de 20 ml,Présentation active,Déclaration de commercialisation,oui,65%,solution injectable,intra-articulaire,Autorisation active,20030101,20020101,Procédure nationale,GUERBET,51.013442
3668,3668_test,plaquette(s) aluminium polyamide PVC-Aluminium...,Présentation active,Déclaration de commercialisation,oui,65%,comprimé gastro-résistant(e),orale,Autorisation active,20120101,20090101,Procédure décentralisée,RANBAXY PHARMACIE GENERIQUES,23.532943
3669,3669_test,1 flacon(s) en verre de 100 ml,Présentation active,Déclaration de commercialisation,oui,65%,solution pour perfusion,intraveineuse,Autorisation active,20140101,20130101,Procédure nationale,BIOGARAN,33.040584


In [71]:
input_df

,drug_id,description,administrative_status,marketing_status,approved_for_hospital_use,reimbursement_rate,dosage_form,route_of_administration,marketing_authorization_status,marketing_declaration_date,marketing_authorization_date,marketing_authorization_process,pharmaceutical_companies
0,0_test,plaquette(s) thermoformée(s) polyamide alumini...,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,comprimé gastro-résistant(e),orale,Autorisation active,20130101,20080101,Procédure de reconnaissance mutuelle,TEVA SANTE
1,1_test,plaquette(s) thermoformée(s) PVC PVDC aluminiu...,Présentation active,Déclaration de commercialisation,non,30%,capsule,vaginale,Autorisation active,20120101,19970101,Procédure nationale,INNOTECH INTERNATIONAL
2,2_test,plaquette(s) thermoformée(s) PVC-Aluminium de ...,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,comprimé,orale,Autorisation abrogée,20130101,20060101,Procédure nationale,ACTAVIS FRANCE
3,3_test,5 cartouche(s) en verre de 3 ml dans stylo pré...,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,suspension injectable,sous-cutanée,Autorisation active,20120101,20000101,Procédure centralisée,SANOFI AVENTIS (ALLEMAGNE)
4,4_test,5 poche(s) à 3 compartiments polyamide polypro...,Présentation active,Déclaration de commercialisation,oui,65%,émulsion pour perfusion,intraveineuse,Autorisation active,20110101,20110101,Procédure décentralisée,B BRAUN MELSUNGEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3666,3666_test,6 ampoule(s) en verre brun de 2 ml,Présentation active,Déclaration de commercialisation,oui,30%,solution injectable,intramusculaire,Autorisation active,20080101,20060101,Procédure nationale,PHARMY II
3667,3667_test,1 seringue(s) préremplie(s) en verre de 20 ml,Présentation active,Déclaration de commercialisation,oui,65%,solution injectable,intra-articulaire,Autorisation active,20030101,20020101,Procédure nationale,GUERBET
3668,3668_test,plaquette(s) aluminium polyamide PVC-Aluminium...,Présentation active,Déclaration de commercialisation,oui,65%,comprimé gastro-résistant(e),orale,Autorisation active,20120101,20090101,Procédure décentralisée,RANBAXY PHARMACIE GENERIQUES
3669,3669_test,1 flacon(s) en verre de 100 ml,Présentation active,Déclaration de commercialisation,oui,65%,solution pour perfusion,intraveineuse,Autorisation active,20140101,20130101,Procédure nationale,BIOGARAN


In [35]:
def generate_batch_predictions(input_df: pd.DataFrame) -> pd.DataFrame:

    """ Crea copia del df original, pasa datos al endpoint en JSON (dict list), recupera predicciones y las une al df original """

    try:
        input_list = input_df.to_dict(orient='records')
        
        API_ENDPOINT = "http://127.0.0.1:8000/batch_predict_pipeline"
        #API_ENDPOINT = "https://3vr4qseoy37cebds3fko6hpldm0xudrw.lambda-url.us-east-2.on.aws/batch_predict_pipeline"
        response = requests.post(API_ENDPOINT, json=input_list)
        
        if response.status_code == 200:
            predictions_output = response.json()
            predictions_df = pd.DataFrame(predictions_output)
        else:
             print("Error al realizar las estimaciones") 

        return predictions_df
    
    except Exception as e:
        logging.error(e)
        raise e
    

In [72]:
input_list = input_df.to_dict(orient='records')

In [73]:
API_ENDPOINT = "http://127.0.0.1:8000/batch_predict_pipeline"

In [74]:
response = requests.post(API_ENDPOINT, json=input_list)

In [75]:
if response.status_code == 200:
    predictions_output = response.json()
    predictions_df = pd.DataFrame(predictions_output)
else:
    print("Error al realizar las estimaciones") 

In [77]:
predictions_df

,drug_id,description,administrative_status,marketing_status,approved_for_hospital_use,reimbursement_rate,dosage_form,route_of_administration,marketing_authorization_status,marketing_declaration_date,marketing_authorization_date,marketing_authorization_process,pharmaceutical_companies,prediction
0,0_test,plaquette(s) thermoformée(s) polyamide alumini...,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,comprimé gastro-résistant(e),orale,Autorisation active,20130101,20080101,Procédure de reconnaissance mutuelle,TEVA SANTE,10.982087
1,1_test,plaquette(s) thermoformée(s) PVC PVDC aluminiu...,Présentation active,Déclaration de commercialisation,non,30%,capsule,vaginale,Autorisation active,20120101,19970101,Procédure nationale,INNOTECH INTERNATIONAL,24.496560
2,2_test,plaquette(s) thermoformée(s) PVC-Aluminium de ...,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,comprimé,orale,Autorisation abrogée,20130101,20060101,Procédure nationale,ACTAVIS FRANCE,38.883877
3,3_test,5 cartouche(s) en verre de 3 ml dans stylo pré...,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,suspension injectable,sous-cutanée,Autorisation active,20120101,20000101,Procédure centralisée,SANOFI AVENTIS (ALLEMAGNE),78.960503
4,4_test,5 poche(s) à 3 compartiments polyamide polypro...,Présentation active,Déclaration de commercialisation,oui,65%,émulsion pour perfusion,intraveineuse,Autorisation active,20110101,20110101,Procédure décentralisée,B BRAUN MELSUNGEN,127.190860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3666,3666_test,6 ampoule(s) en verre brun de 2 ml,Présentation active,Déclaration de commercialisation,oui,30%,solution injectable,intramusculaire,Autorisation active,20080101,20060101,Procédure nationale,PHARMY II,2.561693
3667,3667_test,1 seringue(s) préremplie(s) en verre de 20 ml,Présentation active,Déclaration de commercialisation,oui,65%,solution injectable,intra-articulaire,Autorisation active,20030101,20020101,Procédure nationale,GUERBET,51.013442
3668,3668_test,plaquette(s) aluminium polyamide PVC-Aluminium...,Présentation active,Déclaration de commercialisation,oui,65%,comprimé gastro-résistant(e),orale,Autorisation active,20120101,20090101,Procédure décentralisée,RANBAXY PHARMACIE GENERIQUES,21.091738
3669,3669_test,1 flacon(s) en verre de 100 ml,Présentation active,Déclaration de commercialisation,oui,65%,solution pour perfusion,intraveineuse,Autorisation active,20140101,20130101,Procédure nationale,BIOGARAN,33.040584
